# Example preparation of data before Simulated Annealing and then firing up the algorithm
Here, we prepare the data from CHAMP and NucleaSeq according to the S.I. of our manuscript.

In [1]:
import pandas as pd 
import numpy as np 

import os 
cwd = os.getcwd()
#path = '\\Users\\mdepk\\Documents\\CRISPR_kinetic_model\\'
path=cwd+'\\'

import sys
sys.path.append(path)
sys.path.append(path+"simmulated_Annealing_fit\\")
sys.path.append(path+"model\\")

from preprocessing_data import sequence_averaged_weighted as prepData
from model.CRISPRclass import CRISPR 

import SimmulatedAnnealing_DEP as SAfit
import objective_functions as ObjFunc
from time import time
from datetime import datetime

from importlib import reload
reload(prepData);

## Loading the experimental data
It is less important to figure out exactly how this is done, more important that it produces a sequence averaged file to input in simulated annealing algorithm

In [2]:
'''
Let's prepare a dataset with sequence averaged data for spCas9 NucleaSeq and CHAMP 
(equivalent to that used for manuscript)
'''

dataPath =  path+'data_experiment'
fileNucSeq = os.path.join(dataPath, 'NucleaSeq_dataset.csv')
fileCHAMP = os.path.join(dataPath, 'CHAMP_dataset.csv' )


Cas9 = CRISPR(guide_length=20,
                  PAM_length=3,
                  PAM_sequence='NGG')


targetE = 'GACGCATAAAGATGAGACGCTGG'

NucleaSeqData = prepData.WA_NucleaSeq_DEP(filename=fileNucSeq, Cas=Cas9, on_target=targetE)
CHAMPData = prepData.WA_CHAMP_DEP(filename=fileCHAMP, Cas=Cas9, on_target=targetE)
SeqAvgData = NucleaSeqData.merge(CHAMPData, on=['sequence','mismatch_pattern','mismatch_number']   )


saveFile = os.path.join(dataPath, 'sequence_averaged_CHAMP_NucleaSeq_DEP.csv')
SeqAvgData.to_csv(saveFile, index=False )


# Setting up the simulated annealing run

In [3]:
from objective_functions import chi_sqrd_NucleaSeq_CHAMP as chi2


###################################################################################
# /*   bounds on parameters + initial guess                                       #
#   the latter is actually irrelevant, but just for consistency/nice movies :-) *\#
###################################################################################
# --- bounds on parameters  and initial guess (the latter is actually irrelevant) ---

upper_bnd = [5.] + [10.] * 20 + [5.5] * 20 + [5.] + [3.] + [4.]
lower_bnd = [-1.] + [-10.] * 20 + [4.5] * 20 + [-5.] + [1.] + [-1.]

# upper_bnd = [5.] + [10.] * 20 + [10.] * 20 + [5.] + [3.] + [4.]
# lower_bnd = [-1.] + [-10.] * 20 + [0.] * 20 + [-5.] + [1.] + [-1.]
#initial_guess = [2.] + [0.] * 20 + [5.] * 20 + [0.] + [2.] + [2.]


#######################################################
# /*   Cost function/Objective function for SA-fit  *\#
 #######################################################
# --- You made your objective function, assign it here
# (the funcArgs will be unpacked within your own function, so use the same conventions over here) ---
funcArgs = {}
funcArgs["guide_sequence"] = 'GACGCATAAAGATGAGACGCTGG'
funcArgs["ID_Cas"] = "Clv_Saturated_general_energies_v2"
funcArgs["ID_dCas"] = "general_energies_no_kPR"
funcArgs["Cas"] = CRISPR(guide_length=20,
                         PAM_length=3,
                         PAM_sequence='NGG')
funcArgs["concentration_params"] = 10.  # To get the same as Stijn
myObjectiveFunc = chi2

##############################################
# /*   Input experimental data             *\#
##############################################
dataPath = os.path.join(path,'data_experiment')
fileName = os.path.join(dataPath, 'sequence_averaged_CHAMP_NucleaSeq_DEP.csv')  #<---- This is where our seqeunced average file is loaded
SeqAvgData = pd.read_csv(fileName)

xdata = np.array(SeqAvgData['sequence'])

y1 = np.array(SeqAvgData['cleavage_rate_log10'])
y2 = np.array(SeqAvgData['dissociation_const_log_e'])
ydata = np.array(list(zip(y1, y2)))

e1 = np.array(SeqAvgData['cleavage_rate_error_log10'])
e2 = np.array(SeqAvgData['dissociation_const_error_log_e'])
yerr = np.array(list(zip(e1, e2)))


# generate a run definition from time and date

In [3]:
now = datetime.now() # current date and time
run_def= now.strftime("%Y%m%d-%H%M")
print(run_def)

20200921-1953


In [4]:
run_def="20200921-1953"

In [7]:
fileResults = path+"DEP\\Run_"+run_def+"_Search_State.txt"
print(fileResults)

C:\Users\mdepk\Documents\CRISPR_kinetic_model\DEP\Run_20200921-1953_Search_State.txt


## Running an optimization from start

In [ ]:
##############################################
# /*   Call the Simulated Annealing code   *\#
##############################################

# The parameters are here stored as an array as follows:
# \Delta F_PAM, \Delta F_1, ...,\Delta F_20, \Delta Fmm_1, ..., \Delta Fmm_20, log10(kon), log10(kf), log10(kcat)
#                   ontarget energies       |       mismatch penalties      | on rate, forward rate, cleavage rate

# --- unpack the command line arguments ---

fileResults = path+"DEP\\Run_"+run_def+"_Search_State.txt"
fileMonitor = path+"DEP\\Run_"+run_def+"_Search_Monitor.txt"
fileInitMonitor = path+"DEP\\Run_"+run_def+"_Initiation_Monitor.txt"

initial_guess = [2.] + list(1*(np.random.rand(20)-0.5)) + list(5+1*(np.random.rand(20)-0.5)) + [0.] + [2.] + [2.]
initial_guess = [2.] + [0.] * 20 + [5.] * 20 + [0.] + [2.] + [2.]

t1 = time()
best_fit = SAfit.sim_anneal_fit_DEP(xdata=xdata,
                                ydata=ydata,
                                yerr=yerr,
                                Xstart=initial_guess,
                                lwrbnd=lower_bnd,
                                upbnd=upper_bnd,
                                objective_function=myObjectiveFunc,
                                objective_function_args=funcArgs,
                                Tstart=400000.,  #Ball park estimate from test run (just for speed up)
                                delta=0.2,
                                delta_DEP=0.8,
                                tol=1E-5,
                                Tfinal=0.0,
                                adjust_factor=1.3,
                                cooling_rate=0.99,
                                N_int=100,
                                AR_low=40,
                                AR_high=60,
                                use_multiprocessing=True,
                                nprocs=19,  # HPC05 has computers with 20 cores. 1 core reserved for THIS function
                                output_file_results=fileResults,
                                output_file_monitor=fileMonitor,
                                output_file_init_monitor=fileInitMonitor,InitEQ=True, ReStart=False)

t2 = time()

# the following line will get printed into a specific text file when running on cluster.
print("Time elapsed: ", (t2-t1)/3600., "hrs")



> Starting initiation loop the barrier-buster moves
>- acceptence  35.0 % at temperature 400000.0  and barrier-busting power  0.8
>- acceptence  39.0 % at temperature 520000.0  and barrier-busting power  0.8
>- acceptence  47.0 % at temperature 676000.0  and barrier-busting power  0.8

> Starting initiation loop for the free-energy moves
>- acceptence  34.0 % at temperature  676000.0  and step  0.2
>- acceptence  20.0 % at temperature  676000.0  and step  0.15384615384615385
>- acceptence  30.0 % at temperature  676000.0  and step  0.1183431952662722
>- acceptence  40.0 % at temperature  676000.0  and step  0.09103322712790168

> Initiating cool-down at
>--temp:   676000.0 
>--energy step:   0.09103322712790168 
>--bb power step:   0.8
------------------------------------------------------------------------------------------------------------------------------------
|-(over)writing run to C:\Users\mdepk\Documents\CRISPR_kinetic_model\DEP\Run_20200921-1953_Search_State.txt
 /-acceptan

C:\Users\mdepk\Documents\CRISPR_kinetic_model\simmulated_Annealing_fit\SimmulatedAnnealing_DEP.py:519: RuntimeWarning: invalid value encountered in double_scalars
  tolerance_low_enough = abs(SA.ReldE) / SA.average_energy < SA.Tolerance
C:\Users\mdepk\Documents\CRISPR_kinetic_model\simmulated_Annealing_fit\SimmulatedAnnealing_DEP.py:539: RuntimeWarning: invalid value encountered in double_scalars
  SA.Monitor['(last recorded) relative change average energy'] = abs(SA.average_energy - Eavg) / SA.average_energy


 /-acceptance  46.0 % at energy-step size  0.09103322712790168
/--acceptance  48.0 %  at barrier-busting power  0.8
|--Time cost  4.463866198062897 minutes
<--Equilibration reached. Updating and saving search state at T= 669240.0  and chi2= 24754424.264102515
 /-acceptance  37.0 % at energy-step size  0.09103322712790168
/--acceptance  56.00000000000001 %  at barrier-busting power  0.8
|--Time cost  4.538910480340322 minutes
 /-acceptance  54.0 % at energy-step size  0.07002555932915513
/--acceptance  57.99999999999999 %  at barrier-busting power  0.8
|--Time cost  4.159367656707763 minutes
<--Equilibration reached. Updating and saving search state at T= 662547.6  and chi2= 22116735.19316066
 /-acceptance  48.0 % at energy-step size  0.07002555932915513
/--acceptance  55.00000000000001 %  at barrier-busting power  0.8
|--Time cost  4.059824240207672 minutes
<--Equilibration reached. Updating and saving search state at T= 655922.124  and chi2= 19125473.091540154
 /-acceptance  46.0 % at

## Restarting an aborted optimization run

In [ ]:
t1 = time()

fileResults = path+"DEP\\Run_"+run_def+"_Search_State.txt"
fileMonitor = path+"DEP\\Run_"+run_def+"_Search_Monitor.txt"
fileInitMonitor = path+"DEP\\Run_"+run_def+"_Initiation_Monitor.txt"

#fileResults = path+"DEP\\test_Results_bb.txt"
#fileMonitor = path+"DEP\\test_Monitor_bb.txt"
#fileInitMonitor = path+"DEP\\test_InitMonitor_bb.txt"

best_fit = SAfit.restart_SA_DEP(xdata=xdata,
                                ydata=ydata,
                                yerr=yerr,
                                lwrbnd=lower_bnd,
                                upbnd=upper_bnd,
                                objective_function=myObjectiveFunc,
                                objective_function_args=funcArgs,
                                tol=1E-5,
                                Tfinal=0.0,
                                adjust_factor=1.3,
                                cooling_rate=0.99,
                                N_int=400,
                                AR_low=40,
                                AR_high=60,
                                use_multiprocessing=True,
                                nprocs=19,  # HPC05 has computers with 20 cores. 1 core reserved for THIS function
                                output_file_results=fileResults,
                                output_file_monitor=fileMonitor,
                                output_file_init_monitor=fileInitMonitor)

t2 = time()

# the following line will get printed into a specific text file when running on cluster.
print("Time elapsed: ", (t2-t1)/3600., "hrs")


> Initiating cool-down at
>--temp:   3572683.8443474737 
>--energy step:   2.055950550877243e-05 
>--bb power step:   0.020318098186031492
------------------------------------------------------------------------------------------------------------------------------------
|-appending run to C:\Users\mdepk\Documents\CRISPR_kinetic_model\DEP\Run_20200920-1312_Search_State.txt
 /-acceptance  2.75 % at energy-step size  2.055950550877243e-05
/--acceptance  38.25 %  at barrier-busting power  0.020318098186031492
|--Time cost  38.565847754478455 minutes
 /-acceptance  1.7500000000000002 % at energy-step size  1.5815004237517252e-05
/--acceptance  36.75 %  at barrier-busting power  0.015629306296947303
|--Time cost  38.77649845679601 minutes
 /-acceptance  1.5 % at energy-step size  1.2165387875013271e-05
/--acceptance  32.75 %  at barrier-busting power  0.012022543305344078
|--Time cost  39.41862743695577 minutes
 /-acceptance  1.5 % at energy-step size  9.357990673087132e-06
/--acceptance  

## Loading some fits to see how it went

In [ ]:
#the on-target 
targetE = 'GACGCATAAAGATGAGACGCTGG'

file_parameters =   '/Users/mdepk/Documents/CRISPR_kinetic_model/DEP/test_Results_old.txt'

fatch_solution="best_solution"

epsilon_list= []

for n in 0:89:
        Cas_params, dCas_params, epsilon, forward_rates, kon, kf, kcat= kinpar.kinetic_parameters(file_parameters,
                                                                                          ID_Cas="Clv_Saturated_general_energies_v2",
                                                                                          ID_dCas="general_energies_no_kPR",
                                                                                          concentration_nM=10.,
                                                                                          nmbr_fit_params=44)
        epsilon_list.append(epsilon)

landscape_on_target = free_energy_landscape.plot_landscape(guide=targetE,target=targetE,epsilon=epsilon, 
                                                          Cas=Cas9,show_plot=True);

In [ ]:
!True

In [ ]:
fig, ((ax1,ax2))=plt.sub